In [44]:
import numpy as np
from numpy import array
from numpy import sum 
from numpy import matmul
from numpy import multiply
from numpy import dot
from numpy import exp
from numpy import log
from numpy import transpose
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time

#Loading in the MNIST data
file = loadmat('/Users/samsuidman/Desktop/neurophysics/machine_learning/mnistAll.mat') 

#Preparing the data
train_images = file['mnist']['train_images'][0][0]
train_labels = file['mnist']['train_labels'][0][0].transpose()[0]
test_images = file['mnist']['test_images'][0][0]
test_labels = file['mnist']['test_labels'][0][0].transpose()[0]
indices_3 = np.where(train_labels==3)[0]
indices_7 = np.where(train_labels==7)[0]
indices_3_test = np.where(test_labels==3)[0]
indices_7_test = np.where(test_labels==7)[0]
X3 = train_images[:,:,indices_3]
X7 = train_images[:,:,indices_7]
X3_test = test_images[:,:,indices_3_test]
X7_test = test_images[:,:,indices_7_test]
n3 = np.size(X3,2)
n7 = np.size(X7,2)
n3_test = np.size(X3_test,2)
n7_test = np.size(X7_test,2)
X3 = np.reshape(X3,[784,n3])
X7 = np.reshape(X7,[784,n7])
X3_test = np.reshape(X3_test,[784,n3_test])
X7_test = np.reshape(X7_test,[784,n7_test])
X3 = X3/np.max((np.max(np.concatenate(X3)),np.max(np.concatenate(X7))))
X7 = X7/np.max((np.max(np.concatenate(X3)),np.max(np.concatenate(X7))))
X3_test = X3_test/np.max((np.max(np.concatenate(X3_test)),np.max(np.concatenate(X7_test))))
X7_test = X7_test/np.max((np.max(np.concatenate(X3_test)),np.max(np.concatenate(X7_test))))
X3 = (np.insert(X3,0,1,axis=0)).transpose() #shape = (6131,785)
X7 = (np.insert(X7,0,1,axis=0)).transpose() #shape = (6265,785)
X3_test = (np.insert(X3_test,0,1,axis=0)).transpose() #shape = (1010,785)
X7_test = (np.insert(X7_test,0,1,axis=0)).transpose() #shape = (1028,785)
t3 = np.zeros([1,n3])[0] #length = 6131
t7 = np.ones([1,n7])[0] #length = 6265
t3_test = np.zeros([1,n3_test])[0] #length = 1010
t7_test = np.ones([1,n7_test])[0] #length = 1028

#These are the important matrices and arrays in the end. X3 consists of P=6131 images of the number 3. X[3] for example is the 4th image. Each image has 28x28=784 pixels. To make sure that an image doesn't only consists of zeros, before each image is a 1 added. 
X = np.concatenate([X3,X7]) #shape = (12396,785)
t = np.concatenate([t3,t7]) #shape = (12396,)
X_test = np.concatenate([X3_test,X7_test]) #shape = (2038,785)
t_test = np.concatenate([t3_test,t7_test]) #shape = (2038,)





#these are the functions that are defined in the exercise 

def y(w,X): #this function takes a random w and a matrix with N patterns and d dimensions (in our case 785 pixels in total) and returns an array that contains for each pattern the probability that the number is a 3 (or 7)
    y = 1/(1+exp(-dot(X,w)))
    return y 

def E(w,X,t): #Return the error for w,X and the actual value 3,7 (so actual 1,0)
    y0 = y(w,X)
    Ew = -sum(t*log(y0)+(1-t)*log(1-y0))/len(X)
    return Ew

def dE(w,X,t): #gradient for w,X,t
    y0 = y(w,X)
    dE = matmul((y0-t),X)/len(X)
    return dE

def H(w,X,t): #Hessian for w,X,t
    y0 = y(w,X) 
    X_y = transpose(multiply(transpose(X),y0*(1-y0))) #multiply each pattern in X by the y value of that pattern. This results in a matrix. 
    H = matmul(transpose(X_y),X) #multiply this X*y times X, and sum over all patterns. The new shape is then (785,785)
    return H

def dE_weight_decay(w,X,t,k): #Gradient for the weight decay excercises
    y0 = y(w,X)
    Ew = -sum(t*log(y0)+(1-t)*log(1-y0)+k/len(w)*w)/len(X)
    return Ew

def H_weight_decay(w,X,t,k): #Hessian for the weight decay exercises
    y0 = y(w,X) 
    X_y = transpose(multiply(transpose(X),y0*(1-y0))) #multiply each pattern in X by the y value of that pattern. This results in a matrix. 
    H = matmul(transpose(X_y),X) + k/len(w0)*np.eye(len(w0)) #multiply this X*y times X, and sum over all patterns. The new shape is then (785,785)
    return H



#These are other helpful functions

def wrong_patterns(w,X,t): #Takes a model w, a dataset X and labels t and return the fraction of misclassified patterns
    y0 = y(w,X)
    classifications = np.where(y0>0.5,1,0)
    wrong_predictions = np.sum(classifications!=t)
    wrong_fraction = wrong_predictions/len(X)
    return wrong_fraction


def visualize(w,X,i): #Takes a model w, a dataset X and a image number and visualizes the image with what the model thinks it represents
    digit = int(np.where(y(w,X[i])>0.5,7,3))
    plt.imshow(X[i][1:].reshape(28,28))
    return digit





#These are the learning algorithms for the exercise 

def grad_descent(w0,X,t,e,runs): #The gradient descent learning algorithm 
    T1 = time.time()
    w = w0.copy()
    for i in range(runs):
        w += -e*dE(w,X,t)
        if i%100==0:
            T2 = time.time()
            print(i,str(round(T2-T1,1))+'s','E='+str(round(E(w,X,t),3)))
    return w

def momentum(w0,X,t,e,a,runs):
    T1 = time.time()
    w = w0.copy()
    dw_old = 0
    for i in range(runs):
        dw_new = -e*dE(w,X,t) + a*dw_old
        w += dw_new + dw_old
        dw_old = dw_new.copy()
        if i%100==0:
            T2 = time.time()
            print(i,str(round(T2-T1,1))+'s','E='+str(round(E(w,X,t),3)))
    return w

def weight_decay(w0,X,t,e,a,k,runs): 
    T1 = time.time()
    w = w0.copy()
    dw_old = 0
    for i in range(runs):
        dw_new = -e*dE(w,X,t) + a*dw_old
        w += dw_new + dw_old
        dw_old = dw_new.copy()
        if i%100==0:
            T2 = time.time()
            print(i,str(round(T2-T1,1))+'s','E='+str(round(E(w,X,t),3)))
    return w



#This is executing the script 
w0 = np.random.normal(0,1,size=X.shape[1]) #initializing a random model w0 



In [45]:
w_weight_decay = weight_decay(w0,X,t,e=0.05,a=0.03,k=0.01,runs=10000) #learning the model via weight decay 
results_weight_decay = [[E(w_weight_decay,X,t),wrong_patterns(w_weight_decay,X,t)],[E(w_weight_decay,X_test,t_test),wrong_patterns(w_weight_decay,X_test,t_test)]]




0 0.0s E=6.189
100 1.5s E=0.314
200 2.7s E=0.208
300 4.0s E=0.17
400 5.3s E=0.148
500 6.6s E=0.135
600 7.8s E=0.125
700 9.0s E=0.118
800 10.2s E=0.111
900 11.6s E=0.106
1000 12.9s E=0.102
1100 14.1s E=0.098
1200 15.2s E=0.095
1300 16.3s E=0.092
1400 17.4s E=0.089
1500 18.7s E=0.087
1600 19.9s E=0.085
1700 21.0s E=0.083
1800 22.4s E=0.081
1900 23.6s E=0.079
2000 24.7s E=0.078
2100 25.8s E=0.076
2200 27.0s E=0.075
2300 28.6s E=0.074
2400 29.7s E=0.072
2500 30.8s E=0.071
2600 32.0s E=0.07
2700 33.1s E=0.069
2800 34.3s E=0.068
2900 35.4s E=0.067
3000 36.4s E=0.066
3100 37.6s E=0.065
3200 38.6s E=0.065
3300 39.7s E=0.064
3400 41.0s E=0.063
3500 42.0s E=0.062
3600 43.0s E=0.062
3700 44.0s E=0.061
3800 45.0s E=0.061
3900 46.0s E=0.06
4000 47.1s E=0.059
4100 48.1s E=0.059
4200 49.1s E=0.058
4300 50.2s E=0.058
4400 51.5s E=0.057
4500 52.8s E=0.057
4600 54.2s E=0.056
4700 55.4s E=0.056
4800 56.4s E=0.055
4900 57.4s E=0.055
5000 58.5s E=0.055
5100 59.5s E=0.054
5200 60.6s E=0.054
5300 61.7s E=0.0

In [46]:
results_weight_decay

[[0.04211489763596055, 0.012423362374959664],
 [0.07219452850755184, 0.020117762512266928]]